In [ ]:
pip install selenium beautifulsoup4 pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 12.4 MB/s eta 0:00:00


In [ ]:
import random
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

# Настройки Chrome
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36")

driver = webdriver.Chrome(options=options)

# --- Собираем ссылки на заведения со всех страниц ---
all_firm_links = set()
for page in range(1, 3):  # 2 страниц
    print(f"📄 Загружаем страницу {page}...")
    search_url = f"https://2gis.kz/almaty/search/рестораны и кафе/rubricId/161/page/{page}/?m=76.903857,43.246691/14.35"
    driver.get(search_url)
    time.sleep(5)

    # Прокрутка вниз
    for _ in range(3):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

    soup = BeautifulSoup(driver.page_source, "html.parser")
    place_links = soup.find_all("a", href=True)
    firm_links = [
        "https://2gis.kz" + link['href']
        for link in place_links
        if "/firm/" in link['href'] and "/tab" not in link['href']
    ]
    all_firm_links.update(firm_links)

# Выбор 20 случайных заведений
unique_links = list(all_firm_links)
random.shuffle(unique_links)
selected_links = unique_links[:25]
print(f"\n🔎 Найдено {len(unique_links)} заведений. Выбрано {len(selected_links)} случайных для парсинга.")

# --- Парсим отзывы по заведениям ---
reviews_data = []

for i, place_url in enumerate(selected_links, start=1):
    print(f"\n📍 Обрабатывается {i}/{len(selected_links)}: {place_url}")
    try:
        driver.get(place_url)
        time.sleep(5)

        # Переход на вкладку "Отзывы"
        try:
            review_tab = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, "//a[contains(@href, '/tab/reviews')]"))
            )
            review_tab.click()
            time.sleep(5)
        except:
            print("⚠ Нет вкладки 'Отзывы', пропускаем.")
            continue

        # Название заведения
        soup = BeautifulSoup(driver.page_source, "html.parser")
        name_tag = soup.find("h1", class_="_1x89xo5")
        name = name_tag.get_text(strip=True) if name_tag else "Не найдено"

        # Средняя оценка и количество оценок
        try:
            rating_value = driver.find_element(By.CLASS_NAME, "_y10azs").text.strip()
        except:
            rating_value = "Нет данных"
        try:
            rating_count = driver.find_element(By.CLASS_NAME, "_jspzdm").text.strip().split(" ")[0]
        except:
            rating_count = "Нет данных"

        # Прокрутка для подгрузки отзывов
        for _ in range(10):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)

        # Отзывы (до 50)
        review_blocks = driver.find_elements(By.XPATH, "//div[@class='_1k5soqfl']")[:50]
        for block in review_blocks:
            try:
                user = block.find_element(By.XPATH, ".//span[@class='_16s5yj36']").get_attribute("title")
            except:
                user = "Не указано"
            try:
                date = block.find_element(By.XPATH, ".//div[@class='_a5f6uz']").text
            except:
                date = "Не указана"
            try:
                comment = block.find_element(By.XPATH, ".//div[@class='_49x36f']").text
            except:
                comment = "Комментарий отсутствует"
            try:
                stars = block.find_elements(By.XPATH, ".//div[@class='_1fkin5c']/span")
                rating = len(stars)
            except:
                rating = "Не оценено"

            reviews_data.append({
                "Пользователь": user,
                "Дата": date,
                "Комментарий": comment,
                "Рейтинг": rating,
                "Название заведения": name,
                "Средняя оценка": rating_value,
                "Кол-во оценок": rating_count
            })

    except Exception as e:
        print(f"❌ Ошибка при обработке {place_url}: {e}")
        continue

# --- Сохраняем результат ---
data = pd.DataFrame(reviews_data)
data.to_csv("top_reviews.csv", index=False, encoding='utf-8-sig')
print(f"\n✅ Парсинг завершен! Сохранено {len(data)} отзывов.")
driver.quit()

📄 Загружаем страницу 1...
📄 Загружаем страницу 2...

🔎 Найдено 24 заведений. Выбрано 24 случайных для парсинга.

📍 Обрабатывается 1/24: https://2gis.kz/almaty/firm/70000001089596089?stat=eyJwYXJlbnRUYWJJZCI6IjA1ZTU1ZjVhLWM2YzUtNDRjYi05MzIzLTFkMzc4YTg5ZGIyZiIsImZvcmtFdmVudE9yZGluYWwiOjIsInVpRWxlbWVudCI6eyJuYW1lIjoicGxhY2VDYXJkTWluaSIsIm93bmVyTmFtZSI6InNlYXJjaFJlc3VsdHNMaXN0IiwicG9zaXRpb24iOjE0fSwicGxhY2VJdGVtIjp7ImVudGl0eSI6eyJpZCI6IjcwMDAwMDAxMDg5NTk2MDg5IiwidHlwZSI6ImJyYW5jaCIsInNlZ21lbnRJbmZvIjp7ImJhc2VMb2NhbGUiOiJydV9LWiIsInNlZ21lbnRJZCI6IjY3In19LCJiZXN0S2V5d29yZCI6ItCg0LXRgdGC0L7RgNCw0L3RiyDQuCDQutCw0YTQtSIsImdlb1Bvc2l0aW9uIjp7ImxvbiI6NzYuOTIyNjU0LCJsYXQiOjQzLjIzNTc5OX0sImFkc1NlYXJjaCI6dHJ1ZSwibWFpblJ1YnJpYyI6IjE2MSIsImlzRGVsZXRlZCI6ZmFsc2UsIm9yZyI6IjcwMDAwMDAxMDg1NjcxOTQwIiwiY29udGV4dFJ1YnJpYyI6IjE2MSIsImhhc01vZGVsIjpmYWxzZSwicmVzdWx0Q2xhc3MiOjIsInBvc2l0aW9uIjoxNH0sInNlYXJjaFJlc3VsdHNMaXN0Ijp7InNpemUiOjI2ODUsInNlYXJjaEhhc2giOiJlZDhmZTMyOC1jODkwLTRlYWQtOWZkNy0wZTliOWI0ZWRiMzciLCJ0e

In [ ]:
dd = pd.read_csv("top_reviews_файл.csv")
dd

,Пользователь,Дата,Комментарий,Рейтинг,Название заведения,Средняя оценка,Кол-во оценок
0,Aygul Kudaybergenova,сегодня,Классно,5,Grand Park,4.6,23313
1,Saparbek Turdunmatov,18 апреля 2025,Намазхана жоқ 😕,1,Grand Park,4.6,23313
2,Ришат URN,"18 апреля 2025, отредактирован",Почти всё с Пекина. Расположение бутиков мне н...,3,Grand Park,4.6,23313
3,Alpysbay ✨,"18 апреля 2025, отредактирован","Очень хороший парк, большой и близко:)",5,Grand Park,4.6,23313
4,Ержан Сырымов,"17 апреля 2025, отредактирован",За последний часто посещаем данный ТРЦ. Очень ...,5,Grand Park,4.6,23313
...,...,...,...,...,...,...,...
475,Alimahkmad Hkalifa email,14 апреля 2025,Все супер,5,Almaty Mall,4.6,7569
476,White Heart,"14 апреля 2025, отредактирован",Нормально,5,Almaty Mall,4.6,7569
477,Bek A,"14 апреля 2025, отредактирован",Ланжоу лапша не вкусная,1,Almaty Mall,4.6,7569
478,Алия Шакимова,14 апреля 2025,Маған ұнайды Алматы Молл,5,Almaty Mall,4.6,7569
